In [94]:
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from datetime import datetime
import pandas as pd

x = [1, 2, 4, 5, 7, 8, 9]
x = [datetime(2022, 1, xx) for xx in x]
y = [0, 0.003, 0.01, 0.03, 0.06, 0.12, 0.11]
cats = [[2], [2, 4], [4], [5], [2, 3, 4, 5], [1]]

n_points = 1000

cat_lut = {
    1: "Nothing",
    2: "Beer",
    3: "Wine",
    4: "Liquor (Drinks)",
    5: "Liquor (Shots)"
}
color_lut = {
    "Nothing": "#0F0E0E",
    "Beer": "#f28e1c",
    "Wine": "#BA091E",
    "Liquor (Drinks)": "#ADD8E6",
    "Liquor (Shots)": "#2364AA"
}

possible_cats = len(cat_lut)

xx, yy, cc = [], [], []
for idx, ((x0, x1), (y0, y1)) in enumerate(zip(zip(x[:-1], x[1:]), zip(y[:-1], y[1:]))):
    points = int(n_points*((x1 - x0)/(max(x) - min(x))))
    xxe = pd.to_datetime(np.linspace(pd.Timestamp(x0).value, pd.Timestamp(x1).value, points))
    xx.extend(pd.to_datetime(np.linspace(pd.Timestamp(x0).value, pd.Timestamp(x1).value, points)))
    yy.extend(np.linspace(y0, y1, points))
    colors = [val for val in list(cats[idx]) for _ in range(int(points/len(cats[idx])))]
    colors = colors + [cats[idx][-1]]*(points - len(colors))
    cc.extend(colors)
    
cc = [cat_lut[e] if e in cat_lut else e for e in cc]

fig = px.scatter(x=xx, y=yy, color=cc,
                 color_discrete_map=color_lut)
fig.show()

125
250
125
250
125
125


In [59]:
cc

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 [],
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,

AttributeError: 'list' object has no attribute 'replace'